<a href="https://colab.research.google.com/github/Hlompho-Dash/first_try_of_fastai/blob/master/02_fully_connected.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

from google.colab import drive
drive.mount('/content/gdrive')

os.chdir('/content/gdrive/MyDrive/first_try_of_fastai')

print("------------------------------------------------------------------")

Mounted at /content/gdrive
------------------------------------------------------------------


In [2]:
print(os.getcwd())

/content/gdrive/MyDrive/first_try_of_fastai


In [3]:
#export

from exp.nb_01 import *

def get_data():
  path2 = "/content/gdrive/MyDrive/first_try_of_fastai/mnist (1).pkl.gz"
  with gzip.open(path2, "rb") as f:
    ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding = "latin-1")
  return map(tensor, (x_train,y_train,x_valid,y_valid))
def normalize(x, m, s): return (x-m)/s

In [4]:
x_train, y_train, x_valid, y_valid = get_data()

In [5]:
train_mean, train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [6]:
x_train = normalize(x_train, train_mean, train_std)

x_valid = normalize(x_valid, train_mean, train_std)

In [7]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(3.8966e-08), tensor(1.))

In [8]:
#export

def test_near_zero(a, tol=1e-3):  assert a.abs()<tol, f"Near zero: {a}"

In [9]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [10]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

**Basic architecture**

In [11]:
nh = 50

In [12]:
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh )
b2 = torch.zeros(1)

In [13]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [14]:
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [15]:
def lin(x, w, b): return x@w + b

In [16]:
t = lin(x_valid, w1, b1)
t.mean(),t.std()

(tensor(0.0162), tensor(1.0555))

In [17]:
def relu(x): return x.clamp_min(0.)

In [18]:
t = relu(lin(x_valid, w1, b1))

In [19]:
t.mean(),t.std()

(tensor(0.4290), tensor(0.6206))

**  FIXING THE VANISHING GRADIENT PROBLEM**

In [20]:
w1 = torch.rand(m,nh)*math.sqrt(2/m)
w1.mean(),w1.std()

(tensor(0.0252), tensor(0.0146))

In [21]:
t= relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(1.0099), tensor(1.6895))

In [22]:
 #export

 from torch.nn import init

In [23]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode = "fan_out")

t = relu(lin(x_valid, w1, b1))

In [24]:
init.kaiming_normal_??

In [25]:
w1.mean(),w1.std()

(tensor(0.0002), tensor(0.0504))

In [26]:
t.mean(),t.std()

(tensor(0.4670), tensor(0.7673))

In [27]:
w1.shape

torch.Size([784, 50])

In [28]:
import torch.nn

In [29]:
torch.nn.Linear.forward??

In [30]:
torch.nn.functional.linear??

In [31]:
torch.nn.Conv2d??

In [32]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [33]:
def relu(x): return x.clamp_min(0.) - 0.5

In [34]:
w1 = torch.randn(m,nh)*math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t.std()

(tensor(0.0689), tensor(0.7673))

In [35]:
def model(xb):
  l1 = lin(xb, w1, b1)
  l2 = relu(l1)
  l3 = lin(l2, w2, b2)

  return l3

In [36]:
%timeit -n 10 _= model(x_valid)

10 loops, best of 3: 22.5 ms per loop


In [37]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0],1])

In [44]:
x_valid.shape

torch.Size([10000, 784])

In [38]:
torch.Size([x_valid.shape[0],1])

torch.Size([10000, 1])

In [43]:
model(x_valid).shape

torch.Size([10000, 1])

**LOSS FUNCTION: MSE**

In [46]:
def mse(output, target): return (output.squeeze(-1) - target).pow(2).mean()

In [47]:
y_train, y_valid = y_train.float(), y_valid.float()

In [48]:
preds = model(x_train)

In [49]:
preds.shape

torch.Size([50000, 1])

In [50]:
mse(preds, y_train)

tensor(31.8522)

**GRADIENTS AND BACKWARD PASSES**

In [57]:
def mse_grad(inp, targ):
  # grad of loss with respect to output
  inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1)/ inp.shape[0]

In [58]:
def relu_grad(inp, out):
   # grad of relu with respect to input activations
   inp.g = (inp > 0).float() * out.g

In [59]:
def lin_grad(inp, out, w, b):
  # grad of matmul with respect to input
  inp.g = out.g @ w.t()
  w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
  b.g = out.g.sum(0)

In [60]:
def forward_and_backward(inp, targ):
  # forward pass:
  l1 = inp @ w1 + b1
  l2 = relu(l1)
  out = l2 @ w2 + b2

  loss = mse(out, targ)

  #backward pass:
  mse_grad(out, targ)
  lin_grad(l2, out, w2, b2)
  relu_grad(l1, l2)
  lin_grad(inp, l1, w1, b1)


In [61]:
forward_and_backward(x_train, y_train)

In [62]:
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [71]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)


In [72]:
def forward(inp, targ):
  # forward
  l1 = inp @ w12 + b12
  l2 = relu(l1)
  out = l2 @ w22 + b22

  return mse(out,targ)

In [69]:
loss = forward(xt2, y_train)

In [84]:
# THIS CELL HAS PROBLEMS, LOOK INTO THEM
"""test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig)"""

'test_near(w22.grad, w2g)\ntest_near(b22.grad, b2g)\ntest_near(w12.grad, w1g)\ntest_near(b12.grad, b1g)\ntest_near(xt2.grad, ig)'

BEFORE WE GO INTO REFACTORING THE MODEL NOTE THE FOLLOWING:

1. MATRIX REPRESENTATIONS OF NEURAL NETS. NOT QUITE CLEAR ON THAT
2. THE ".g" OF TENSORS IN THE GRADIENT CALCULATIONS ON THE BACKPROP
3. LEAST THEN LOOK INTO THE SMALL TEST FRAMEWORKS BUILT IN

***REFACTORING THE MODEL***

*LAYERS AS CLASSES*

In [86]:
class Relu():
  def __call__(self, inp):
    self.inp = inp
    self.out = inp.clamp_min(0.) - 0.5
    
    return self.out

  def backward(self): self.inp.g = (self.inp > 0).float() * self.out.g

In [87]:
class Lin():
  def __init__(self,w,b):
    self.w, self.b = w,b
  
  def __call__(self,inp):
    self.inp = inp
    self.out = inp@self.w + self.b
    return self.out

  def backward(self):
    self.inp.g = self.out.g @ self.w.t()
    self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
    self.b.g = self.out.g.sum(0)

In [90]:
class Mse():
  def __call__(self,inp,targ):
    self.inp = inp
    self.targ = targ
    self.out = (inp.squeeze() - targ).pow(2).mean()

  def backward(self):
    self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [94]:
class Model():
  def __init__(self, w1, b1, w2, b2):
    self.layers = [Lin(w1,b1),Relu(), Lin(w2,b2)]
    self.loss = Mse()

  def __call__(self, x , targ):
    for l in self.layers: x = l(x)
    return self.loss(x, targ)

  def backward(self):
    self.loss.backward()
    for l in reversed(self.layers): l.backward()

In [95]:
w1.g,b1.g,w2.g,b2.g = [None]*4

model = Model(w1, b1, w2, b2)

In [96]:
%time loss = model(x_train, y_train)

CPU times: user 217 ms, sys: 862 µs, total: 217 ms
Wall time: 226 ms


In [97]:
%time model.backward()

CPU times: user 4.02 s, sys: 8.34 s, total: 12.4 s
Wall time: 12.4 s


In [99]:
# initialization and calling, different parameters to be taken
Model(w1, b1, w2, b2)(x_train, y_train)

CPU times: user 127 ms, sys: 0 ns, total: 127 ms
Wall time: 130 ms
